<a href="https://colab.research.google.com/github/tobby-lie/Recurrent-Neural-Network-from-Scratch/blob/master/Lie_5931_PA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import csv
import itertools
import operator
import numpy as np
import sys
from datetime import datetime
import matplotlib.pyplot as plt

%cd /content/drive/My\ Drive/5931_PA3
%ls

!pip3 install nltk

/content/drive/My Drive/5931_PA3
Lie_5931_PA3.ipynb  SciFi_Three.txt


In [4]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Unzipping tokenizers/punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [0]:
vocabulary_size = 8000
unknown_token = "UNKNOWN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
num_ascii_char = 256

In [6]:
print("hello")

hello


In [7]:
print("Reading .txt file...")

sentences = []
sentence_list = []
filename = "SciFi_Three.txt"
with open(filename, "r") as f:
    
    all_content = f.read()
    paragraphs = all_content.split("\n\n")
    
    for paragraph in paragraphs:
        
        #if cleanedLine: # is not empty
        #sentences.append(line)

        sentences = nltk.sent_tokenize(paragraph.lower())
        sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]

        for sentence in sentences:
            sentence = sentence.replace("\n", " ")
            sentence_list.append(sentence)
            
print("Parsed %d sentences." % (len(sentence_list)))
            

Reading .txt file...
Parsed 25324 sentences.


In [8]:
sentence_list[0:100]

['SENTENCE_START the mysterious island SENTENCE_END',
 'SENTENCE_START by jules verne SENTENCE_END',
 'SENTENCE_START 1874 SENTENCE_END',
 'SENTENCE_START  part 1--dropped from the clouds SENTENCE_END',
 'SENTENCE_START chapter 1 SENTENCE_END',
 'SENTENCE_START “are we rising again?” “no. SENTENCE_END',
 'SENTENCE_START on the contrary.” “are we descending?”  “worse than that, captain! SENTENCE_END',
 'SENTENCE_START we are falling!” “for heaven’s sake heave out the ballast!” “there! SENTENCE_END',
 'SENTENCE_START the last sack is empty!” “does the balloon rise?”  “no!” “i hear a noise like the dashing of waves. SENTENCE_END',
 'SENTENCE_START the sea is below the car! SENTENCE_END',
 'SENTENCE_START it cannot be more than 500 feet from us!” “overboard with every weight! SENTENCE_END',
 'SENTENCE_START ... everything!” SENTENCE_END',
 'SENTENCE_START such were the loud and startling words which resounded through the air, above the vast watery desert of the pacific, about four o’clock 

In [0]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentence_list]

In [10]:
len(tokenized_sentences)

25324

In [11]:
tokenized_sentences[6]

['SENTENCE_START',
 'on',
 'the',
 'contrary.',
 '”',
 '“',
 'are',
 'we',
 'descending',
 '?',
 '”',
 '“',
 'worse',
 'than',
 'that',
 ',',
 'captain',
 '!',
 'SENTENCE_END']

In [0]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))

In [13]:
print("Found %d unique words tokens." % len(word_freq.items()))

Found 20063 unique words tokens.


In [14]:
word_freq

FreqDist({'SENTENCE_START': 25324,
          'the': 31822,
          'mysterious': 56,
          'island': 843,
          'SENTENCE_END': 25324,
          'by': 2691,
          'jules': 6,
          'verne': 28,
          '1874': 1,
          'part': 303,
          '1': 34,
          '--': 977,
          'dropped': 22,
          'from': 2128,
          'clouds': 64,
          'chapter': 199,
          '“': 6188,
          'are': 931,
          'we': 2674,
          'rising': 87,
          'again': 311,
          '?': 2099,
          '”': 6109,
          'no': 1519,
          '.': 19243,
          'on': 3123,
          'contrary.': 2,
          'descending': 11,
          'worse': 18,
          'than': 638,
          'that': 4591,
          ',': 35583,
          'captain': 1083,
          '!': 2757,
          'falling': 43,
          'for': 2439,
          'heaven': 32,
          '’': 4039,
          's': 1756,
          'sake': 9,
          'heave': 2,
          'out': 688,
          '

In [0]:
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [16]:
word_to_index

{',': 0,
 'the': 1,
 'SENTENCE_START': 2,
 'SENTENCE_END': 3,
 '.': 4,
 'of': 5,
 'to': 6,
 'and': 7,
 'a': 8,
 'in': 9,
 '“': 10,
 '”': 11,
 'was': 12,
 'i': 13,
 'it': 14,
 'that': 15,
 '’': 16,
 'had': 17,
 'this': 18,
 'on': 19,
 'which': 20,
 'he': 21,
 'with': 22,
 'at': 23,
 '!': 24,
 'his': 25,
 'by': 26,
 'we': 27,
 'not': 28,
 'as': 29,
 'but': 30,
 'be': 31,
 'for': 32,
 'my': 33,
 'were': 34,
 'from': 35,
 'they': 36,
 'is': 37,
 '?': 38,
 's': 39,
 'have': 40,
 'you': 41,
 ';': 42,
 'would': 43,
 'no': 44,
 'an': 45,
 'all': 46,
 'their': 47,
 'our': 48,
 "''": 49,
 '``': 50,
 'its': 51,
 'could': 52,
 'there': 53,
 'then': 54,
 'these': 55,
 'one': 56,
 'been': 57,
 'me': 58,
 'some': 59,
 'said': 60,
 'if': 61,
 'so': 62,
 'captain': 63,
 'pencroft': 64,
 'what': 65,
 'or': 66,
 'him': 67,
 't': 68,
 '--': 69,
 'them': 70,
 'are': 71,
 'us': 72,
 'will': 73,
 'harding': 74,
 'more': 75,
 'into': 76,
 'island': 77,
 'when': 78,
 'up': 79,
 'replied': 80,
 'sea': 81,
 'two

In [17]:
print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))
word_freq.most_common(10)

Using vocabulary size 8000.
The least frequent word in our vocabulary is 'sober' and appeared 3 times.


[(',', 35583),
 ('the', 31822),
 ('SENTENCE_START', 25324),
 ('SENTENCE_END', 25324),
 ('.', 19243),
 ('of', 14395),
 ('to', 11175),
 ('and', 10695),
 ('a', 8523),
 ('in', 6697)]

In [18]:
index_to_word[7999]

'UNKNOWN'

In [0]:
# Replace all words not in vocabulary with the unknown token
for i, sentence in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sentence]

In [20]:
print("\nExample sentence: '%s'" % sentence_list[0])
print("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])


Example sentence: 'SENTENCE_START the mysterious island SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'the', 'mysterious', 'island', 'SENTENCE_END']'


In [0]:
# Create the training data
XTrain = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
YTrain = np.asarray([[word_to_index[w] for w in sent [1:]] for sent in tokenized_sentences])



In [22]:
XTrain

array([list([2, 1, 843, 77]), list([2, 26, 4825, 1537]), list([2, 7999]),
       ...,
       list([2, 299, 16, 68, 13, 1161, 343, 541, 5, 18, 7999, 590, 38]),
       list([2, 347, 6, 15, 313, 146, 7006, 419, 1495, 9, 1, 1262, 5, 7999, 10, 85, 104, 5657, 1, 7999, 425, 38, 11, 7999, 269, 87, 5, 46, 1703, 40, 84, 4347, 1, 179, 6, 392, 4]),
       list([2, 63, 106, 7, 5246])], dtype=object)

In [23]:
YTrain

array([list([1, 843, 77, 3]), list([26, 4825, 1537, 3]), list([7999, 3]),
       ...,
       list([299, 16, 68, 13, 1161, 343, 541, 5, 18, 7999, 590, 38, 3]),
       list([347, 6, 15, 313, 146, 7006, 419, 1495, 9, 1, 1262, 5, 7999, 10, 85, 104, 5657, 1, 7999, 425, 38, 11, 7999, 269, 87, 5, 46, 1703, 40, 84, 4347, 1, 179, 6, 392, 4, 3]),
       list([63, 106, 7, 5246, 3])], dtype=object)

In [24]:
import random

for i in range(5):
    idx = random.randint(0, len(XTrain))
    x_example, y_example = XTrain[idx], YTrain[idx]
    print("Sample id %d\n========================\n" % idx)
    print("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example))
    print("\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example))

Sample id 21077

x:
SENTENCE_START “ ned my friend , ” conseil replied , “ had you ever heard of the nautilus ?
[2, 10, 125, 33, 455, 0, 11, 120, 80, 0, 10, 17, 41, 311, 278, 5, 1, 89, 38]

y:
“ ned my friend , ” conseil replied , “ had you ever heard of the nautilus ? SENTENCE_END
[10, 125, 33, 455, 0, 11, 120, 80, 0, 10, 17, 41, 311, 278, 5, 1, 89, 38, 3]
Sample id 3153

x:
SENTENCE_START no mercy was shown them .
[2, 44, 334, 12, 2209, 70, 4]

y:
no mercy was shown them . SENTENCE_END
[44, 334, 12, 2209, 70, 4, 3]
Sample id 12860

x:
SENTENCE_START the great granite tunnel , as it went round by UNKNOWN and winding ways , presented every now and then sharp turns , and in fact all the appearance of a labyrinth .
[2, 1, 118, 122, 951, 0, 29, 14, 159, 461, 26, 7999, 7, 2181, 2655, 0, 1212, 157, 84, 7, 54, 832, 2350, 0, 7, 9, 297, 46, 1, 702, 5, 8, 3390, 4]

y:
the great granite tunnel , as it went round by UNKNOWN and winding ways , presented every now and then sharp turns , and in fact

In [0]:
class RNNVanilla:

    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):

        # Assign instance variables
        self.word_dim = word_dim # size of the vocabulary
        self.hidden_dim = hidden_dim # size of hidden layer
        self.bptt_truncate = bptt_truncate

        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))



In [0]:
def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

In [0]:
def forward_propagation(self, x):
    # Total number of time steps
    T = len(x)

    # During forward propagation we save all hidden states in s because we need them later.

    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)

    # The outputs at each time step, Again, we save them for later
    o = np.zeros((T, self.word_dim))

    # For each time step...
    for t in np.arange(T):
        # Note that we are indexing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:, x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s] # We not only return the calculated outputs, but also the hidden states.
                  # We will use them later to calculate the gradients

# Now make it a member of the class RNNVanilla
RNNVanilla.forward_propagation = forward_propagation

In [0]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

# Now make it a member of the class RNNVanilla
RNNVanilla.predict = predict

In [29]:
print("Length of XTrain[10] is %d" % (len(XTrain[10])))
np.random.seed(100)

model = RNNVanilla(vocabulary_size)
o, s = model.forward_propagation(XTrain[10])
print(o.shape)
print(o)
# For each word in the sentence, the model made 8000 predictions representing probabilities of the next word
# Note that because we initialized U, V, W to random values these predictions are completely random right now

Length of XTrain[10] is 19
(19, 8000)
[[0.0001246  0.00012553 0.00012589 ... 0.00012485 0.00012517 0.00012529]
 [0.00012536 0.00012431 0.00012473 ... 0.00012514 0.00012464 0.00012557]
 [0.00012478 0.00012452 0.00012558 ... 0.00012454 0.00012574 0.00012446]
 ...
 [0.00012477 0.00012464 0.00012533 ... 0.00012504 0.00012468 0.00012499]
 [0.00012518 0.00012559 0.00012501 ... 0.00012528 0.00012452 0.0001252 ]
 [0.00012431 0.00012588 0.00012578 ... 0.00012629 0.00012445 0.00012479]]


In [30]:
# The following gives the indices of the highest probability predictions for each word:
predictions = model.predict(XTrain[10])
print(predictions.shape)
print(predictions)
print("index_to_word>")
print('%s'%" ".join([index_to_word[x] for x in predictions]))

(19,)
[1386 2542 1193 7993 3728 6162 6969 2924 6734 5804 7288 3712 6787 4191
   67 1409 6734 6695 5419]
index_to_word>
expect liked worth 1219 & anywhere expert navigation mounting weakened lion outline veil loudly him king mounting occurring recollect


In [0]:
def calculate_total_loss(self, x, y):
    L = 0

    # For each sentence
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])

        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]

        # Add to the loss based on how off we were
        L += -1 * sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N
    
RNNVanilla.calculate_total_loss = calculate_total_loss
RNNVanilla.calculate_loss = calculate_loss
    

In [32]:
np.log(vocabulary_size)


8.987196820661973

In [33]:
XTrain[:100].shape

(100,)

In [34]:
XTrain.shape

(25324,)

In [35]:
# Limit to 1000 examples to save time
print("Expected Loss for random predictions: %f" % np.log(vocabulary_size))
print("Actual loss: %f" % model.calculate_loss(XTrain[:1000], YTrain[:1000]))


Expected Loss for random predictions: 8.987197
Actual loss: 8.987239


In [0]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)

        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))

        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:

            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t

            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]
    

RNNVanilla.bptt = bptt       

In [0]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to check if these are correct.
    bptt_gradients = self.bptt(x, y)

    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']

    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print("Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape)))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = self.calculate_total_loss([x], [y])
            parameter[ix] = original_value - h
            gradminus = self.calculate_total_loss([x], [y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print ("Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix))
                print ("+h Loss: %f" % gradplus)
                print ("-h Loss: %f" % gradminus)
                print ("Estimated_gradient: %f" % estimated_gradient)
                print ("Backpropagation gradient: %f" % backprop_gradient)
                print ("Relative Error: %f" % relative_error)
                return
            it.iternext()
        print ("Gradient check for parameter %s passed." % (pname))

RNNVanilla.gradient_check = gradient_check

In [38]:
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNVanilla(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [0]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW
    
RNNVanilla.sgd_step = numpy_sdg_step

In [0]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print ("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5 
                print ("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1
    return losses

In [41]:
np.random.seed(10)
model = RNNVanilla(vocabulary_size)
%timeit model.sgd_step(XTrain[10], YTrain[10], learning_rate=0.005)

10 loops, best of 3: 70.1 ms per loop


In [78]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNVanilla(vocabulary_size)
losses = train_with_sgd(model, XTrain[:1000], YTrain[:1000], nepoch=10, evaluate_loss_after=1)

2019-10-28 03:38:19: Loss after num_examples_seen=0 epoch=0: 8.987061
2019-10-28 03:40:06: Loss after num_examples_seen=1000 epoch=1: 6.000864
2019-10-28 03:41:49: Loss after num_examples_seen=2000 epoch=2: 5.700542
2019-10-28 03:43:30: Loss after num_examples_seen=3000 epoch=3: 5.537883
2019-10-28 03:45:08: Loss after num_examples_seen=4000 epoch=4: 5.416308
2019-10-28 03:46:45: Loss after num_examples_seen=5000 epoch=5: 5.311065
2019-10-28 03:48:22: Loss after num_examples_seen=6000 epoch=6: 5.216516
2019-10-28 03:50:02: Loss after num_examples_seen=7000 epoch=7: 5.138017
2019-10-28 03:51:40: Loss after num_examples_seen=8000 epoch=8: 5.074504
2019-10-28 03:53:19: Loss after num_examples_seen=9000 epoch=9: 5.025341


In [80]:
for loss in losses:
    print(loss)

(0, 8.987061217422854)
(1000, 6.000864145514668)
(2000, 5.700541547115805)
(3000, 5.5378828034649805)
(4000, 5.4163076679444835)
(5000, 5.311064936565079)
(6000, 5.216516457128669)
(7000, 5.138016974710205)
(8000, 5.0745041073157235)
(9000, 5.02534138640177)


In [82]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[0][-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str
 
num_sentences = 10
senten_min_length = 7
 
for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print (" ".join(sent))

it had all the space fear a spilett which the open .
administered consequently just brave waterspouts by the coast through the grouse thermometer the prisoners several his gesture , his undulating !
he ’ on chimneys inside , in a fire he him him a alarm give him , ” that a general ’ down him .
on the last could be two to chimneys , they hold approaching to than our hard .
the reporter must spilett and s at the sea him hope give zeal , plumage handfuls above his give , “ ? , the sea was been intend .
“ during influence his could struck by when in our try .
he besides off dissolving is over perhaps it arrived soul some he had there steer went waistcoat shouting from the first .
“ if a perhaps besides rag that it when too , and up some importance to be eyes .
and the few was all dwelling on the sprung cyrus the rooted .
capital a so alone the opening , and then .
